<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from collections import Counter
from sklearn.metrics.scorer import make_scorer
from sklearn.dummy import DummyRegressor

In [2]:
df = pd.read_csv('/datasets/insurance.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [3]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Данные загружены, все на месте, ничего не обычного, двигаемся дальше.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? 

**Ответ:** Я считаю что качество измениться не должно.

**Обоснование:** Лучшее обоснование это конечно эксперимент, но алгебраическое доказательство ниже.


$$
Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w'
$$

$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$
$$
w' = (P^T ((X^T X) P))^{-1} P^T X^T y
$$
$$
w' =  ((X^T X) P)^{-1} ((P^T)^{-1} P^T) X^T y
$$
$$
w' =  ((X^T X) P)^{-1} E X^T y
$$
$$
w' = P^{-1} (X^T X)^{-1} X^T y
$$
$$
w' = P^{-1} w
$$

## Алгоритм преобразования

**Алгоритм**

1) Мы берем случайную матрицу. 

2) Проверяем ее на обратимость чтобы можно было с ее помощью вернуть изначальные значения признаков. 

3) Делим признаки на обучающую и валидационную выборки.

4) Обучаем модель линейной регрессии, смотрим результат метрикой r2.

5) Затем создаем новую переменную в которую мы записываем признаки и умножаем на нашу случайную обратимую матрицу.

6) С помощью измененных признаков обучаем вторую модель, смотрим результат метрикой r2.

7) Делаем вывод.

**Обоснование**

Если при умножении любой случайной обратимой матрицы качество линейной регрессии не изменится, значит таким способом мы можем зашифровать данные, а потом вернуть их с помощью умножения на матрицу обратную случайной.

## Проверка алгоритма

In [5]:
random_matrix = np.random.normal(size = (4, 4))
random_matrix

array([[-0.57586712,  2.09442337, -0.3375425 ,  0.51657602],
       [-0.52817254, -0.64251412, -0.17407295,  0.80356828],
       [-0.85498276,  0.52756981,  0.49049313, -0.27391597],
       [-0.55383492,  0.51445362, -0.48333966, -0.72406901]])

In [6]:
reverse_random_matrix = np.linalg.inv(random_matrix)
reverse_random_matrix

array([[ 0.06015656, -0.60188091, -0.57411577, -0.40785822],
       [ 0.39221588, -0.34947214,  0.0287292 , -0.11889032],
       [-0.13627544, -0.40415881,  0.92057679, -0.89401217],
       [ 0.32362579,  0.48186213, -0.15496599, -0.5568059 ]])

In [7]:
features = df.drop('Страховые выплаты', axis = 1)
target = df['Страховые выплаты']

In [8]:
features @ random_matrix

,0,1,2,3
0,-42429.929880,26143.728582,24320.501334,-13553.493504
1,-32514.194811,20018.611743,18630.248213,-10372.566940
2,-17969.955052,11060.333188,10295.307597,-5728.931974
3,-35664.980560,21987.197362,20448.941275,-11406.869320
4,-22330.414844,13753.676178,12796.659086,-7126.190433
...,...,...,...,...
4995,-30538.781182,18817.280878,17504.763990,-9757.748498
4996,-44819.608545,27623.327237,25695.438149,-14326.599785
4997,-28995.586827,17872.795326,16623.268941,-9271.128291
4998,-27971.793557,17241.035397,16033.508158,-8941.029479


In [9]:
(features @ random_matrix) @ reverse_random_matrix

,0,1,2,3
0,1.000000e+00,41.0,49600.0,1.000000e+00
1,-1.273083e-12,46.0,38000.0,1.000000e+00
2,2.123755e-13,29.0,21000.0,-1.178164e-12
3,-1.453131e-13,21.0,41700.0,2.000000e+00
4,1.000000e+00,28.0,26100.0,-1.796983e-12
...,...,...,...,...
4995,1.134388e-12,28.0,35700.0,2.000000e+00
4996,-1.455763e-12,34.0,52400.0,1.000000e+00
4997,-7.939378e-15,20.0,33900.0,2.000000e+00
4998,1.000000e+00,22.0,32700.0,3.000000e+00


In [10]:
features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                       target, test_size = 0.25, random_state=12345)

In [11]:
model = LinearRegression()
model.fit(features_train, target_train)
answer = model.predict(features_valid)
r2_score(answer, target_valid)

-0.2869270212348376

In [12]:
features_train2 = features_train @ random_matrix
features_valid2 = features_valid @ random_matrix

In [13]:
model2 = LinearRegression()
model2.fit(features_train2, target_train)
answer2 = model2.predict(features_valid2)
r2_score(answer2, target_valid)

-0.2869270212348052

### Вывод: 
Мы видим что качество линейной регрессии не изменилось до 13го знака, и мы можем защифровать данные таким способом.